# IFRS9 Risk System - Exploratory Data Analysis

This notebook provides exploratory data analysis of the synthetic loan portfolio data.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Configure plotting
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
%matplotlib inline

## 1. Data Loading and Overview

In [ ]:
# Generate synthetic data if not exists
import sys
sys.path.append('/home/jovyan/src')

from generate_data import DataGenerator

# Generate sample data
generator = DataGenerator(seed=42)
loans_df = generator.generate_loan_portfolio(n_loans=1000)
payments_df = generator.generate_payment_history(loans_df, n_months=12)
macro_df = generator.generate_macroeconomic_data()

print(f"Loans: {len(loans_df):,} records")
print(f"Payments: {len(payments_df):,} records")
print(f"Macro data: {len(macro_df):,} records")

In [ ]:
# Basic info about the loan portfolio
print("Loan Portfolio Overview:")
print(f"Total loans: {len(loans_df):,}")
print(f"Total exposure: ${loans_df['current_balance'].sum():,.2f}")
print(f"Total ECL: ${loans_df['ecl_amount'].sum():,.2f}")
print(f"Coverage ratio: {loans_df['ecl_amount'].sum() / loans_df['current_balance'].sum():.2%}")

loans_df.head()

## 2. IFRS9 Stage Distribution

In [ ]:
# Stage distribution
stage_dist = loans_df['provision_stage'].value_counts()
print("Stage Distribution:")
for stage, count in stage_dist.items():
    pct = count / len(loans_df) * 100
    print(f"{stage}: {count:,} ({pct:.1f}%)")

# Visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Count distribution
stage_dist.plot(kind='bar', ax=ax1, color=['green', 'orange', 'red'])
ax1.set_title('Loan Count by IFRS9 Stage')
ax1.set_ylabel('Number of Loans')
ax1.tick_params(axis='x', rotation=0)

# Exposure distribution
stage_exposure = loans_df.groupby('provision_stage')['current_balance'].sum()
stage_exposure.plot(kind='bar', ax=ax2, color=['green', 'orange', 'red'])
ax2.set_title('Exposure by IFRS9 Stage')
ax2.set_ylabel('Total Exposure ($)')
ax2.tick_params(axis='x', rotation=0)

plt.tight_layout()
plt.show()

## 3. Credit Risk Analysis

In [ ]:
# Credit score distribution by stage
plt.figure(figsize=(12, 8))

for i, stage in enumerate(['STAGE_1', 'STAGE_2', 'STAGE_3'], 1):
    plt.subplot(2, 2, i)
    data = loans_df[loans_df['provision_stage'] == stage]['credit_score']
    plt.hist(data, bins=30, alpha=0.7, edgecolor='black')
    plt.title(f'{stage} Credit Score Distribution')
    plt.xlabel('Credit Score')
    plt.ylabel('Count')

# Combined distribution
plt.subplot(2, 2, 4)
for stage in ['STAGE_1', 'STAGE_2', 'STAGE_3']:
    data = loans_df[loans_df['provision_stage'] == stage]['credit_score']
    plt.hist(data, bins=30, alpha=0.6, label=stage, edgecolor='black')
plt.title('Credit Score Distribution by Stage')
plt.xlabel('Credit Score')
plt.ylabel('Count')
plt.legend()

plt.tight_layout()
plt.show()

## 4. Expected Credit Loss Analysis

In [ ]:
# ECL analysis by loan type and stage
ecl_analysis = loans_df.groupby(['loan_type', 'provision_stage']).agg({
    'ecl_amount': ['sum', 'mean', 'count'],
    'current_balance': 'sum',
    'pd_rate': 'mean',
    'lgd_rate': 'mean'
}).round(4)

print("ECL Analysis by Loan Type and Stage:")
print(ecl_analysis)

# ECL by loan type visualization
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Total ECL by loan type
ecl_by_type = loans_df.groupby('loan_type')['ecl_amount'].sum().sort_values(ascending=False)
ecl_by_type.plot(kind='bar', ax=axes[0,0], color='skyblue')
axes[0,0].set_title('Total ECL by Loan Type')
axes[0,0].set_ylabel('ECL Amount ($)')
axes[0,0].tick_params(axis='x', rotation=45)

# Average PD by loan type and stage
pd_pivot = loans_df.pivot_table(values='pd_rate', index='loan_type', columns='provision_stage', aggfunc='mean')
pd_pivot.plot(kind='bar', ax=axes[0,1], color=['green', 'orange', 'red'])
axes[0,1].set_title('Average PD by Loan Type and Stage')
axes[0,1].set_ylabel('Probability of Default')
axes[0,1].tick_params(axis='x', rotation=45)
axes[0,1].legend(title='Stage')

# Coverage ratio by loan type
coverage_by_type = loans_df.groupby('loan_type').apply(
    lambda x: x['ecl_amount'].sum() / x['current_balance'].sum()
).sort_values(ascending=False)
coverage_by_type.plot(kind='bar', ax=axes[1,0], color='lightcoral')
axes[1,0].set_title('Coverage Ratio by Loan Type')
axes[1,0].set_ylabel('ECL / Exposure')
axes[1,0].tick_params(axis='x', rotation=45)

# ECL distribution
axes[1,1].hist(loans_df['ecl_amount'], bins=50, edgecolor='black', alpha=0.7)
axes[1,1].set_title('ECL Amount Distribution')
axes[1,1].set_xlabel('ECL Amount ($)')
axes[1,1].set_ylabel('Count')
axes[1,1].set_yscale('log')

plt.tight_layout()
plt.show()

## 5. Interactive Plotly Visualizations

In [ ]:
# Interactive scatter plot: Credit Score vs ECL colored by Stage
fig = px.scatter(loans_df, 
                x='credit_score', 
                y='ecl_amount',
                color='provision_stage',
                size='current_balance',
                hover_data=['loan_type', 'pd_rate', 'lgd_rate'],
                title='Credit Score vs ECL by IFRS9 Stage',
                color_discrete_map={'STAGE_1': 'green', 'STAGE_2': 'orange', 'STAGE_3': 'red'})

fig.update_layout(height=600)
fig.show()

In [ ]:
# Portfolio composition sunburst chart
fig = px.sunburst(loans_df, 
                 path=['loan_type', 'provision_stage'], 
                 values='current_balance',
                 title='Portfolio Composition: Loan Type → IFRS9 Stage')

fig.update_layout(height=600)
fig.show()

## 6. Payment Analysis

In [ ]:
# Payment behavior analysis
payment_stats = payments_df.groupby('payment_status').agg({
    'loan_id': 'count',
    'actual_payment': 'sum',
    'scheduled_payment': 'sum'
})

print("Payment Statistics:")
print(payment_stats)

# Payment trends over time
payments_df['payment_date'] = pd.to_datetime(payments_df['payment_date'])
monthly_payments = payments_df.groupby([payments_df['payment_date'].dt.to_period('M'), 'payment_status']).size().unstack(fill_value=0)

monthly_payments.plot(kind='area', stacked=True, figsize=(12, 6), 
                     color=['green', 'red'], alpha=0.7)
plt.title('Payment Status Trends Over Time')
plt.xlabel('Month')
plt.ylabel('Number of Payments')
plt.legend(title='Payment Status')
plt.show()

## 7. Summary and Key Insights

In [ ]:
# Generate summary statistics
print("=" * 60)
print("IFRS9 PORTFOLIO ANALYSIS SUMMARY")
print("=" * 60)

# Portfolio metrics
total_exposure = loans_df['current_balance'].sum()
total_ecl = loans_df['ecl_amount'].sum()
coverage_ratio = total_ecl / total_exposure

print(f"\nPortfolio Overview:")
print(f"  Total Loans: {len(loans_df):,}")
print(f"  Total Exposure: ${total_exposure:,.2f}")
print(f"  Total ECL: ${total_ecl:,.2f}")
print(f"  Coverage Ratio: {coverage_ratio:.2%}")

# Stage distribution
print(f"\nStage Distribution:")
for stage in ['STAGE_1', 'STAGE_2', 'STAGE_3']:
    count = len(loans_df[loans_df['provision_stage'] == stage])
    exposure = loans_df[loans_df['provision_stage'] == stage]['current_balance'].sum()
    ecl = loans_df[loans_df['provision_stage'] == stage]['ecl_amount'].sum()
    print(f"  {stage}: {count:,} loans (${exposure:,.2f} exposure, ${ecl:,.2f} ECL)")

# Risk metrics by loan type
print(f"\nRisk Metrics by Loan Type:")
for loan_type in loans_df['loan_type'].unique():
    subset = loans_df[loans_df['loan_type'] == loan_type]
    avg_pd = subset['pd_rate'].mean()
    avg_lgd = subset['lgd_rate'].mean()
    type_coverage = subset['ecl_amount'].sum() / subset['current_balance'].sum()
    print(f"  {loan_type}: {avg_pd:.2%} PD, {avg_lgd:.2%} LGD, {type_coverage:.2%} Coverage")

print("\n" + "=" * 60)